In [2]:
import configparser
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Configuration
cross_days = ["2024-07-05", "2024-07-06", "2024-07-07", "2024-07-18", "2024-07-19", 
              "2024-07-20", "2024-07-24", "2024-07-25", "2024-07-26", "2024-07-27", 
              "2024-07-28", "2024-07-29", "2024-07-30", "2024-07-31"]
activities = ["toileting", "resting", "exit", "cooking", "shower", "pc", "sleep", "kitchen"]

# Read configuration file
config = configparser.ConfigParser()
config.read('agg-config.txt')

# Helper functions
def day_time(ti):
    return int((int)((ti + off_zone) / (60 * 60 * 24)))

def time2str(tt):
    ttt = int(tt)
    return datetime.fromtimestamp(ttt).strftime("%Y-%m-%d %H:%M:%S")

# Functions to calculate metrics in a range N
def check_accuracy_range(data, N):
    correct_predictions = 0
    for i in range(len(data)):
        ground_truth = data.iloc[i]['ground_truth']
        prediction = data.iloc[i]['prediction']
        if ground_truth == prediction:
            correct_predictions += 1
        else:
            for j in range(max(0, i - N), min(len(data), i + N + 1)):
                if data.iloc[j]['ground_truth'] == prediction:
                    correct_predictions += 1
                    break
    return correct_predictions / len(data)

def check_precision_range(data, N):
    true_positives = 0
    predicted_positives = 0
    for i in range(len(data)):
        prediction = data.iloc[i]['prediction']
        if prediction == 1:
            predicted_positives += 1
            for j in range(max(0, i - N), min(len(data), i + N + 1)):
                if data.iloc[j]['ground_truth'] == prediction:
                    true_positives += 1
                    break
    return true_positives / predicted_positives if predicted_positives > 0 else 0

def check_recall_range(data, N):
    true_positives = 0
    actual_positives = sum(data['ground_truth'])
    for i in range(len(data)):
        ground_truth = data.iloc[i]['ground_truth']
        if ground_truth == 1:
            for j in range(max(0, i - N), min(len(data), i + N + 1)):
                if data.iloc[j]['prediction'] == ground_truth:
                    true_positives += 1
                    break
    return true_positives / actual_positives if actual_positives > 0 else 0

# Main processing loop
users_day = {}
for cross_day in cross_days:
    users_day[cross_day] = config.get('Configuration', cross_day)
    users_day[cross_day] = [item.strip() for item in users_day[cross_day].split(',')]

off_zone = 60 * 60 * 2

for cross_day in cross_days:
    ini_date = cross_day
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")
    next_day_dt = cross_day_dt + timedelta(days=1)
    ini_date_dt = datetime.strptime(cross_day + " 02:00:00", "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(next_day_dt.strftime("%Y-%m-%d") + " 01:59:59", "%Y-%m-%d %H:%M:%S")

    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    t0 = int(datetime.strptime(new_ini_date_str, '%Y-%m-%d %H:%M:%S').timestamp())
    day = day_time(t0)
    print(cross_day + "#" + ("#".join(users_day[cross_day])))

    users = users_day[ini_date]
    for activity in activities:
        for user in users:
            try:
                # Read the data file
                data = pd.read_csv(f"./data-har/DAY_{str(day)}/act/PREDICTION-TRANS2-other-30+15.{user}.{activity}.tsv", 
                                   sep='\t', header=None, names=['time', 'ground_truth', 'score'])
                data['prediction'] = data['score'].apply(lambda x: 1 if x >= 0.5 else 0)

                # Define range N
                N = 3  # Adjust this value as needed
                
                # Calculate metrics
                accuracy = check_accuracy_range(data, N)
                precision = check_precision_range(data, N)
                recall = check_recall_range(data, N)
                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                if precision == 0 and recall == 0:
                    precision = recall = f1 = 1

                # Print results
                print(f"{activity}\t#{user}\t{accuracy:.2f}\t{precision:.2f}\t{recall:.2f}\t{f1:.2f}".replace('.', ','))

            except FileNotFoundError:
                # Skip missing files
                #print(f"File not found for {user} and activity {activity} on {cross_day}. Skipping...")
                continue


2024-07-05#0001#3d57#ed9c
toileting	#0001	0,96	0,49	1,00	0,66
toileting	#3d57	0,99	0,64	1,00	0,78
toileting	#ed9c	0,99	0,75	1,00	0,86
resting	#0001	0,90	0,60	0,88	0,72
resting	#3d57	0,76	0,84	0,45	0,59
resting	#ed9c	0,93	0,10	1,00	0,18
exit	#0001	0,99	0,98	1,00	0,99
exit	#3d57	0,98	0,99	0,96	0,98
exit	#ed9c	0,96	1,00	0,93	0,97
cooking	#0001	0,96	0,76	1,00	0,87
cooking	#3d57	0,97	0,46	1,00	0,63
cooking	#ed9c	0,97	0,38	1,00	0,55
shower	#0001	1,00	1,00	1,00	1,00
shower	#3d57	1,00	0,50	1,00	0,67
shower	#ed9c	1,00	0,43	1,00	0,60
pc	#0001	1,00	1,00	1,00	1,00
pc	#3d57	0,98	0,33	0,42	0,37
pc	#ed9c	1,00	0,99	1,00	1,00
sleep	#0001	1,00	1,00	1,00	1,00
sleep	#3d57	0,97	0,97	0,97	0,97
sleep	#ed9c	0,99	1,00	0,99	1,00
kitchen	#0001	0,95	0,83	0,99	0,90
kitchen	#3d57	0,99	0,77	0,76	0,76
kitchen	#ed9c	0,99	0,50	1,00	0,67
2024-07-06#0001#3d57#ed9c
toileting	#0001	0,98	0,52	1,00	0,68
toileting	#3d57	0,99	0,50	0,93	0,65
toileting	#ed9c	0,97	0,53	1,00	0,69
resting	#0001	0,84	0,20	1,00	0,33
resting	#3d57	0,8